In [1]:
import pyarrow.parquet as pq
import pandas as pd
from IPython.display import display

#dataset_1 = pq.ParquetDataset('dataset-1/w32processes')
#dataset_2 = pq.ParquetDataset('dataset-2/w32processes')
#dataset_3 = pq.ParquetDataset('dataset-3/w32processes')
dataset_4 = pq.ParquetDataset('../dataset-4/w32processes')

#table_1 = dataset_1.read()
#table_2 = dataset_2.read()
#table_3 = dataset_3.read()
table_4 = dataset_4.read()

#df_1 = table_1.to_pandas()
#df_2 = table_2.to_pandas()
#df_3 = table_3.to_pandas()
df = table_4.to_pandas()

#df_1.to_csv('test_1.csv')
#df_2.to_csv('test_2.csv')
#df_3.to_csv('test_3.csv')
#df_4.to_csv('test_4.csv')

pd.set_option('display.max_colwidth', None)

print('Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3')

Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3


In [2]:
#display(df)

In [3]:
n_elevated_processes = df[df['username'].str.contains('NT AUTHORITY')]
print('Nº of elevated processes: ' + str(len(n_elevated_processes)))

Nº of elevated processes: 4346


In [4]:
# Elevated powershell / cmd processes:
elevated_consoles = n_elevated_processes[n_elevated_processes['name'].str.contains('cmd')]
print('Nº of elevated cmds: ' + str(len(elevated_consoles)))

display(elevated_consoles)

Nº of elevated cmds: 8


,hostname,name,arguments,username,path,pid
61,EC0002LKS,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Program Files (x86)\Common Files\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,5968
54,EE0006XXF,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Program Files (x86)\Common Files\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,252
7,NC0009YBY,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,2492
16,SE0006JXV,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,3172
102,SE0006JXV,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Users\masonwood4\Documents\run_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,3424
74,SE0007JXV,cmd.exe,C:\Windows\System32\cmd.exe /C C:\Windows\servicing\LCUrun_pd.bat,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,596
93,SN0007ZVK,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,5368
36,WW0006YQW,cmd.exe,C:\Windows\System32\cmd.exe,NT AUTHORITY\SYSTEM,C:\Windows\System32\cmd.exe,1476


In [5]:
elevated_consoles = n_elevated_processes[n_elevated_processes['name'].str.contains('powershell')]
print('Nº of elevated cmds: ' + str(len(elevated_consoles)))

display(elevated_consoles)

Nº of elevated cmds: 0


,hostname,name,arguments,username,path,pid


In [6]:
# Count on how many different hostnames appears each process
host_count = n_elevated_processes.groupby('name')['hostname'].nunique()

# Rare processes, less than 5 hosts
rare_procs = host_count[host_count < 5].index.tolist()
rare_elevated = n_elevated_processes[n_elevated_processes['name'].isin(rare_procs)]

display(rare_elevated)

print('There are no unique processes appearing in less than 5 hostnames')

,hostname,name,arguments,username,path,pid


There are no unique processes appearing in less than 5 hostnames


In [10]:
# windows services
known_services = [
    'services.exe', 'svchost.exe', 'lsass.exe', 'wininit.exe',
    'taskhostw.exe', 'spoolsv.exe', 'smss.exe', 'csrss.exe',
    'dllhost.exe', 'dcomlaunch.exe', 'winlogon.exe', 'wmiprvse.exe']

service_like = df[
    (df['name'].str.lower().isin(known_services))
]

service_like = service_like[
    service_like['arguments'].notna() & 
    (service_like['arguments'].str.strip() != '')
]

service_like_elevated = service_like[service_like['username'].str.contains('NT AUTHORITY')]

service_host_counts = service_like_elevated.groupby('name')['hostname'].nunique()

rare_services = service_host_counts[service_host_counts < 5].index.tolist()

susp_services = service_like_elevated[service_like_elevated['name'].isin(rare_services)]

display(service_like_elevated)
print(service_host_counts)

,hostname,name,arguments,username,path,pid
0,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k networkservice -p -s Dnscache,NT AUTHORITY\SYSTEM,c:\windows\system32,316
1,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k localserviceandnoimpersonation -p -s SSDPSRV,NT AUTHORITY\SYSTEM,c:\windows\system32,1088
2,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k netsvcs -p -s ProfSvc,NT AUTHORITY\SYSTEM,c:\windows\system32,808
5,EB0005YAT,svchost.exe,c:\windows\system32\svchost.exe -k netsvcs -p,NT AUTHORITY\SYSTEM,c:\windows\system32,2512
6,EB0005YAT,taskhostw.exe,taskhostw.exe {222A245B-E637-4AE9-A93F-A59CA119A75E},NT AUTHORITY\SYSTEM,c:\windows\system32,1764
...,...,...,...,...,...,...
50,WW0006YQW,svchost.exe,C:\Windows\system32\svchost.exe -k DcomLaunch,NT AUTHORITY\SYSTEM,C:\Windows\system32,236
59,WW0006YQW,svchost.exe,C:\Windows\system32\svchost.exe -k LocalServiceNoNetwork,NT AUTHORITY\SYSTEM,C:\Windows\system32,2860
61,WW0006YQW,svchost.exe,C:\Windows\System32\svchost.exe -k LocalSystemNetworkRestricted,NT AUTHORITY\SYSTEM,C:\Windows\System32,3336
62,WW0006YQW,svchost.exe,C:\Windows\System32\svchost.exe -k LocalServicePeerNet,NT AUTHORITY\SYSTEM,C:\Windows\System32,3900


name
DllHost.exe       19
WinLogon.exe      19
csrss.exe         35
dllhost.exe       40
lsass.exe        100
services.exe      35
smss.exe          35
spoolsv.exe      100
svchost.exe      100
taskhostw.exe     46
wininit.exe       54
winlogon.exe      81
wmiprvse.exe      19
Name: hostname, dtype: int64


In [9]:
suspicious_keywords = ['svc', 'service', 'host', 'win', 'sys']
service_like_extra = df[
    df['name'].str.lower().str.contains('|'.join(suspicious_keywords))
]

service_like_extra_elevated = service_like_extra[
    service_like_extra['username'].str.contains('NT AUTHORITY')
]

host_counts_extra = service_like_extra_elevated.groupby('name')['hostname'].nunique()
rare_extra = host_counts_extra[host_counts_extra < 5].index.tolist()

susp_services_extended = service_like_extra_elevated[
    service_like_extra_elevated['name'].isin(rare_extra)
]

all_suspicious_services = pd.concat([susp_services, susp_services_extended]).drop_duplicates()
display(all_suspicious_services)

,hostname,name,arguments,username,path,pid
